In [1]:
import numpy as np
import csv
from collections import OrderedDict
import copy
import pandas as pd
from time import time
from typing import Callable
from typing import Any
from typing import Dict, Tuple, List
import cProfile
import re
import os

In [2]:
t = time()

In [3]:
def readFile(filen: str) -> List[List]:
    '''
    读取文件内容
    由于首先需要获取文章数量和单词向量长度，才能计算TF矩阵
    因此要对文本内容进行两次遍历，为了避免两次读取磁盘文件，故先将文本内容保存到内存中的一个list
    '''
    fdata = []
    with open(filen) as fd:
        reader = csv.reader(fd, delimiter=' ')
        fdata = [list(row) for row in reader]
    return fdata

In [4]:
def getTFIDF(fdata: List[List]) -> np.array:
    '''
    获取TF-IDF矩阵
    '''
    #首先获取文章数和单词向量
    #使用OrderedDict按单词出现的顺序生成单词列表
    #相比于使用list，好处在于每次判断word是否已经加入单词向量是log(n)复杂度
    word_dict = OrderedDict() 
    #文章数
    D = 0
    for row in fdata:
        D += 1
        for word in row:
            if not word in word_dict:
                word_dict[word] = 1
            else:
                word_dict[word] += 1
    #word_vec是单词向量
    word_vec = word_dict.keys()
    #word_order的键值是当前单词的序号，在生成TF矩阵时会用到
    word_order = dict(zip(word_vec,range(len(word_vec))))
    #生成TF矩阵
    TF = np.zeros((D,len(word_dict)))
    for i,row in enumerate(fdata):
        for word in row:
            TF[i][word_order[word]] += 1
        #每个文章中单词出现次数归一化
        TF[i] /= len(fdata[i])
    #生成IDF矩阵
    IDF = np.log2(D / (1 + np.array(list(word_dict.values()))))
    #生成TF-IDF矩阵
    TF_IDF = np.multiply(TF, IDF)
    return TF_IDF

In [5]:

#TF_IDF
print(time()-t)

0.019343137741088867


In [6]:
def KNN_getTFIDF(fdata: List[List], word_dict: OrderedDict) -> np.array:
    '''
    获取TF-IDF矩阵，并将每个单词及出现次数存储到word_dict中
    '''
    #首先获取文章数和单词向量
    #使用OrderedDict按单词出现的顺序生成单词列表
    #相比于使用list，好处在于每次判断word是否已经加入单词向量是log(n)复杂度
    #文章数
    D = len(fdata)
    if len(word_dict) is 0:
        #训练集
        for row in fdata:
            for word in row:
                if not word in word_dict:
                    word_dict[word] = 1
                else:
                    word_dict[word] += 1
        word_dict[None] = 0
    else:
        #验证集和测试集，丢弃未出现的单词
        word_dict = dict(zip(word_dict.keys(), [0 for _ in word_dict.values()]))
        for row in fdata:
            for word in row:
                if word in word_dict:
                    word_dict[word] += 1
                else:
                    word_dict[None] += 1
    #word_vec是单词向量
    word_vec = word_dict.keys()
    #word_order的键值是当前单词的序号，在生成TF矩阵时会用到
    word_order = dict(zip(word_vec,range(len(word_vec))))
    #生成TF矩阵
    TF = np.zeros((D,len(word_dict)))
    for i,row in enumerate(fdata):
        for word in row:
            if word in word_order:
                TF[i][word_order[word]] += 1
            else:
                TF[i][word_order[None]] += 1
        #每个文章中单词出现次数归一化
        TF[i] /= len(fdata[i])
    #生成IDF矩阵
    IDF = np.log2(D / (1 + np.array(list(word_dict.values()))))
    #生成TF-IDF矩阵
    TF_IDF = np.multiply(TF, IDF)
    np.savetxt("foo.csv", TF_IDF, delimiter=",")
    return TF_IDF

In [7]:
wdict = OrderedDict()
KNN_getTFIDF([['a','a'],['c','a']], wdict)


array([[-1. ,  0. ,  0. ],
       [-0.5,  0. ,  0. ]])

In [8]:
def getOneHot(fdata: List[List], word_dict: OrderedDict) -> np.array:
    D = len(fdata)
    if len(word_dict) is 0:
        for row in fdata:
            for word in row:
                if not word in word_dict:
                    word_dict[word] = 1
        word_dict[None] = 0
    else:
        word_dict = dict(zip(word_dict.keys(), [0 for _ in word_dict.values()]))
        for row in fdata:
            for word in row:
                if word in word_dict:
                    word_dict[word] = 1
                else:
                    word_dict[None] = 1
    word_vec = word_dict.keys()
    word_order = dict(zip(word_vec,range(len(word_vec))))
    oneHot = np.zeros((D,len(word_dict)))
    for i,row in enumerate(fdata):
        for word in row:
            if word in word_order:
                oneHot[i][word_order[word]] = 1
            else:
                oneHot[i][word_order[None]] = 1
    return oneHot

In [9]:
wdict = OrderedDict()
getOneHot([['a','a'],['c','a']], wdict)
getOneHot([['d','d']], wdict)

array([[0., 0., 1.]])

In [10]:
def DisN(vec1: np.array, vec2: np.array, N: Any) -> float:
    '''
    计算N-norm
    '''
    if(N < 1):
        raise ValueError("norm should be a positive integer or np.inf")
    if np.isinf(N):
        return np.max(np.fabs(vec1 - vec2))
    else:
        return np.power(np.sum(np.power(vec1 - vec2, N)), 1.0/N)

#Dis2 = lambda v1, v2: DisN(v1, v2, 2)
Dis1 = lambda v1, v2: np.linalg.norm(v1 - v2, 1)
Dis2 = lambda v1, v2: np.linalg.norm(v1 - v2, 2)
DisInf = lambda v1, v2: np.linalg.norm(v1 - v2, np.inf)
def DisCosine(v1, v2):
    t1 = np.dot(v1,v2)
    t2 = np.linalg.norm(v1)
    t3 = np.linalg.norm(v2)
    ret = 1 - t1 / (t2*t3)
    return ret
 

In [11]:
DisCosine([0,1],[0,0.5])

0.0

In [12]:
def DisInvNormAvg(distances: np.array, Y: np.array) -> np.array:
    '''
    将距离倒数归一化，返回均值
    '''
    for idx, dis in enumerate(distances):
        if np.isclose(dis, 0):
            return Y[idx]
    distances = np.array(1.0) / distances
    s = np.sum(distances)
    distances = distances / s
    tmp = np.diag(distances) @ Y  
    if len(tmp.shape) is 1:
        return tmp
    else:
        return np.sum(tmp,  axis = (0))

In [13]:
def KNN(trainSet: Tuple[np.array, np.array],
        testVec: np.array,
        DisFunc: Callable[[np.array, np.array], float],
        K: int,
        WeightFunc: Callable[[np.array, np.array], float]) -> np.array: 
    '''
    一个通用的KNN接口
    trainSet: 二元元组，第一个元素是训练集的X，第二个是Y
    testVec: 待预测向量
    DisFunc: 距离函数
    K: K值
    WeightFunc: 依据第一个参数list<距离>,对第二个参数list<Y值>进行加权，返回预测值
    '''
    #对于多个要预测的值，逐一预测
    if len(testVec.shape) > 1:
        return np.array([KNN(trainSet, vec, DisFunc, K, WeightFunc) for vec in testVec])
    else:
        #测量待预测向量到训练集中每个向量的距离
        #distances是一个list<tuple(index, distance)>
        distances = list(enumerate(map(lambda trainVec: DisFunc(trainVec, testVec), trainSet[0])))
        #依据距离从小到大排序
        distances.sort(key=lambda t: t[1])
        #获取最临近的K个训练样本的下标和对应的距离，输出值
        tmp = list(zip(*distances[:K]))
        kNearIdx = list(tmp[0])
        kNearDis = list(tmp[1])
        kNearY   = trainSet[1][kNearIdx, :]
        #对输出值根据距离加权作为预测输出
        return WeightFunc(kNearDis, kNearY)

In [14]:
def TestCast1():
    trainX = np.array([[10,2],[2,3],[3,5]])
    trainY = np.array([[1,1,1], [2,2,3], [3,3,5]])
    vaildX = np.array([[3,3]])
    return trainX, trainY, vaildX


In [15]:
def TestCast2():
    xfilen = 'lab1_data/X.txt'
    yfilen = 'lab1_data/Y.txt'
    xdata = readFile(xfilen)
    ydata = readFile(yfilen)
    x_set = getTFIDF(xdata)
    y_set = np.array([list(map(float, row)) for row in ydata])
    DIVIDE_RATE = 0.75
    train_D = int(np.ceil(x_set.shape[0] * DIVIDE_RATE))
    trainX = x_set[0:train_D, :]
    vaildX = x_set[train_D:, :]
    trainY = y_set[0:train_D, :]
    vaildY = y_set[train_D:, :]
    return trainX, trainY, vaildX

In [16]:
def Test(trainX, trainY, vaildX):
    t = time()
    cProfile.run('KNN((trainX, trainY), vaildX,Dis2,2,DisInvNormAvg)')
    print(time()-t) 
#trainX, trainY, vaildX = TestCast2()
#Test(trainX, trainY, vaildX)

In [17]:
def classifyReadFile(filen: str) -> Tuple[List[str], List[str]]:
    with open(filen) as fd:
        reader = csv.reader(fd, delimiter=',')
        train_data = [list(row) for row in reader]
        train_data = train_data[1:]
        tmp = list(zip(*train_data))
        xdata = [row.split() for row in list(tmp[0])]
        ydata = list(tmp[1])
        return xdata, ydata

def vectorizeData(xdata, ydata, xVecFunc, yVecFunc):
    return xVecFunc(xdata), yVecFunc(ydata)

def fastHashY(s: str) -> int:
    if s[0] is 'a': return 0
    if s[0] is 'd': return 1
    if s[0] is 'f': return 2
    if s[0] is 'j': return 3
    if s[1] is 'a': return 4
    return 5

def classifyParseY(ydata: List[str])->np.array:
    '''
    Convert Y data from raw string list to matrix consisted of Y vectors
    e.g.
    ["anger", "disgust", ..., "surprise"] -> 
    |1, 0, 0, 0, 0, 0|
    |0, 1, 0, 0, 0, 0|
    |0, 0, ...,  0, 0|
    |0, 0, 0, 0, 1, 0|
    |0, 0, 0, 0, 0, 1|
    '''
    D = len(ydata)
    
    #fast hash ydata from strings ["anger", "disgust", ...] to [1, 2, ...]^T
    ydata = np.array(list(map(fastHashY, ydata))).reshape((-1,1))
    
    '''
    ymat is the column-wise repeat of ydata.
    e.g.
    |0|      |0, 0, 0, 0, 0, 0|
    |1|   -> |1, 1, 1, 1, 1, 1|
    ...      |................|
    |5|      |5, 5, 5, 5, 5, 5|
    ydata -> ymat
    '''
    ymat  = np.tile(ydata, (1, 6))
    
    '''
    ycmp is a matrix of which each row is [0, 1, 2, 3, 4, 5]
    |0, 1, 2, 3, 4, 5|
    |0, 1, 2, 3, 4, 5|
    |................|
    |0, 1, 2, 3, 4, 5|
    '''
    ycmp  = np.tile(np.array(range(6)), (D, 1))
    return np.int_(np.equal(ymat, ycmp))



In [18]:
def do_classify(trainX, trainY, vaildX, vaildY, knnFunc):
    predictY = knnFunc(trainX, trainY, vaildX)
    classifyY = np.zeros_like(predictY)
    for i, row in enumerate(predictY):
        m = 0
        idx = 0
        for j, v in enumerate(row):
            if v > m:
                m = v
                idx = j
        classifyY[i][idx] = 1
    #print("Predicted Y")
    #print(classifyY)
    #print("Correct Y")
    #print(vaildY)
    ret = np.sum(np.logical_and(classifyY, vaildY)) / vaildX.shape[0]
    print("Classification Accuracy: ", ret)
    return ret

In [19]:
def regressReadFile(filen: str) -> Tuple[List[str], List[str]]:
    with open(filen) as fd:
        reader = csv.reader(fd, delimiter=',')
        train_data = [list(row) for row in reader]
        train_data = train_data[1:]
        xdata = [row[0].split() for row in train_data]
        ydata = [[row[i] for i in range(1, 7)] for row in train_data]
        return xdata, ydata

def regressParseY(ydata: List[List[str]]) -> np.array:
    return np.float_(np.array(ydata))

#from scipy.stats.stats import pearsonr
def pearsonr(X, Y):

    X_bar = np.average(X)
    Y_bar = np.average(Y)
    X = np.subtract(X, X_bar)
    Y = np.subtract(Y, Y_bar)
    t1 = np.sum(np.dot(X, Y))
    t2 = np.sum(np.power(X,2))
    t3 = np.sum(np.power(Y,2))

    ret = t1 / np.power(t2 * t3, 0.5)
    return ret
def do_regress(trainX, trainY, vaildX, vaildY, knnFunc, save = False):
    predictY = knnFunc(trainX, trainY, vaildX)
    if save:
        np.savetxt("regress.csv", predictY, delimiter=",", fmt="%4f")
    r = [pearsonr(predictY[:, i], vaildY[:, i]) for i in range(6)]
    average = np.average(r)
    print("Correlation Coefficient: ", average)
    return average
    
    
train_filen = 'lab1_data/regression_dataset/train_set.csv'
vaild_filen = 'lab1_data/regression_dataset/validation_set.csv'
test_filen  = 'lab1_data/regression_dataset/test_set.csv'

In [20]:
#train_filen = 'lab1_data/classification_dataset/train_set.csv'
#vaild_filen = 'lab1_data/classification_dataset/validation_set.csv'
#test_filen  = 'lab1_data/classification_dataset/test_set.csv'
#
#trainX_data, trainY_data = classifyReadFile(train_filen)
#vaildX_data, vaildY_data = classifyReadFile(vaild_filen)
#
#ParseFuncs = {"OneHot": getOneHot, "TI-IDF": KNN_getTFIDF}
#K_val = range(1, 20)
#DisFuncs = {"Dis1": Dis1, "Dis2": Dis2, "DisInf": DisInf, "DisCosine": DisCosine}
#
#results = OrderedDict()
#
#for pfname, ParseFunc in ParseFuncs.items():
#    word_dict = OrderedDict()
#    def classifyParseX(fdata: List[List]): return ParseFunc(fdata, word_dict)
#    trainX, trainY = vectorizeData(trainX_data, trainY_data, classifyParseX, classifyParseY)
#    vaildX, vaildY = vectorizeData(vaildX_data, vaildY_data, classifyParseX, classifyParseY)
#    for K in K_val:
#        for dfname, DisFunc in DisFuncs.items():
#            print("ParseFunc = {}, K = {}, DisFunc = {}".format(pfname, K, dfname))
#            def knnFunc(trainX, trainY, vaildX): return KNN((trainX, trainY), vaildX, DisFunc, K, DisInvNormAvg)
#            ret = do_classify(trainX, trainY, vaildX, vaildY, knnFunc)
#            results[(pfname, K, dfname)] = ret

In [21]:
def autoTrain(train_filen, vaild_filen, ReadFileFunc, ParseYFunc, TrainFunc):
    print("Start training...")
    t = time()
    trainX_data, trainY_data = ReadFileFunc(train_filen)
    vaildX_data, vaildY_data = ReadFileFunc(vaild_filen)
    
    ParseFuncs = {"OneHot": getOneHot, "TI-IDF": KNN_getTFIDF}
    K_val = range(1, 20)
    DisFuncs = {"Dis1": Dis1, "Dis2": Dis2, "DisInf": DisInf, "DisCosine": DisCosine}
    #DisFuncs = {"DisCosine": DisCosine}
    results = OrderedDict()
    
    for pfname, ParseFunc in ParseFuncs.items():
        word_dict = OrderedDict()
        def ParseXFunc(fdata: List[List]): return ParseFunc(fdata, word_dict)
        trainX, trainY = vectorizeData(trainX_data, trainY_data, ParseXFunc, ParseYFunc)
        vaildX, vaildY = vectorizeData(vaildX_data, vaildY_data, ParseXFunc, ParseYFunc)
        for K in K_val:
            for dfname, DisFunc in DisFuncs.items():
                print("ParseFunc = {}, K = {}, DisFunc = {}".format(pfname, K, dfname))
                def knnFunc(trainX, trainY, vaildX): return KNN((trainX, trainY), vaildX, DisFunc, K, DisInvNormAvg)
                ret = TrainFunc(trainX, trainY, vaildX, vaildY, knnFunc)
                results[(pfname, K, dfname)] = ret
    print("{} groups of argument tested, spent {}s".format(len(ParseFuncs) * len(K_val) * len(DisFuncs), time() - t))
    return results

In [22]:
train_filen = 'lab1_data/regression_dataset/train_set.csv'
vaild_filen = 'lab1_data/regression_dataset/validation_set.csv'
test_filen  = 'lab1_data/regression_dataset/test_set.csv'
np.set_printoptions(threshold=np.nan)
regressResults = autoTrain(train_filen, vaild_filen, regressReadFile, regressParseY, do_regress)




Start training...
ParseFunc = OneHot, K = 1, DisFunc = Dis1
Correlation Coefficient:  0.19479721872198139
ParseFunc = OneHot, K = 1, DisFunc = Dis2
Correlation Coefficient:  0.19479721872198139
ParseFunc = OneHot, K = 1, DisFunc = DisInf
Correlation Coefficient:  0.1171662427880124
ParseFunc = OneHot, K = 1, DisFunc = DisCosine
Correlation Coefficient:  0.26806497858040573
ParseFunc = OneHot, K = 2, DisFunc = Dis1
Correlation Coefficient:  0.2193244476147608
ParseFunc = OneHot, K = 2, DisFunc = Dis2
Correlation Coefficient:  0.21658011165364377
ParseFunc = OneHot, K = 2, DisFunc = DisInf
Correlation Coefficient:  0.13936439828225422
ParseFunc = OneHot, K = 2, DisFunc = DisCosine
Correlation Coefficient:  0.33605079445508984
ParseFunc = OneHot, K = 3, DisFunc = Dis1
Correlation Coefficient:  0.28654256615037427
ParseFunc = OneHot, K = 3, DisFunc = Dis2
Correlation Coefficient:  0.2828362544506024
ParseFunc = OneHot, K = 3, DisFunc = DisInf
Correlation Coefficient:  0.14406034627349598
P

Correlation Coefficient:  0.39398895043848653
ParseFunc = TI-IDF, K = 5, DisFunc = Dis1
Correlation Coefficient:  0.32268102227291745
ParseFunc = TI-IDF, K = 5, DisFunc = Dis2
Correlation Coefficient:  0.24491494460993168
ParseFunc = TI-IDF, K = 5, DisFunc = DisInf
Correlation Coefficient:  0.14701527336696027
ParseFunc = TI-IDF, K = 5, DisFunc = DisCosine
Correlation Coefficient:  0.40148953705180057
ParseFunc = TI-IDF, K = 6, DisFunc = Dis1
Correlation Coefficient:  0.3225732962609001
ParseFunc = TI-IDF, K = 6, DisFunc = Dis2
Correlation Coefficient:  0.24678062172184898
ParseFunc = TI-IDF, K = 6, DisFunc = DisInf
Correlation Coefficient:  0.1580745395658509
ParseFunc = TI-IDF, K = 6, DisFunc = DisCosine
Correlation Coefficient:  0.3980207315613325
ParseFunc = TI-IDF, K = 7, DisFunc = Dis1
Correlation Coefficient:  0.31902210008896553
ParseFunc = TI-IDF, K = 7, DisFunc = Dis2
Correlation Coefficient:  0.25307409218967486
ParseFunc = TI-IDF, K = 7, DisFunc = DisInf
Correlation Coeffic

In [23]:
train_filen = 'lab1_data/classification_dataset/train_set.csv'
vaild_filen = 'lab1_data/classification_dataset/validation_set.csv'
test_filen  = 'lab1_data/classification_dataset/test_set.csv'
classifyResults = autoTrain(train_filen, vaild_filen, classifyReadFile, classifyParseY, do_classify)



Start training...
ParseFunc = OneHot, K = 1, DisFunc = Dis1
Classification Accuracy:  0.3665594855305466
ParseFunc = OneHot, K = 1, DisFunc = Dis2
Classification Accuracy:  0.3665594855305466
ParseFunc = OneHot, K = 1, DisFunc = DisInf
Classification Accuracy:  0.3729903536977492
ParseFunc = OneHot, K = 1, DisFunc = DisCosine
Classification Accuracy:  0.39228295819935693
ParseFunc = OneHot, K = 2, DisFunc = Dis1
Classification Accuracy:  0.31189710610932475
ParseFunc = OneHot, K = 2, DisFunc = Dis2
Classification Accuracy:  0.31189710610932475
ParseFunc = OneHot, K = 2, DisFunc = DisInf
Classification Accuracy:  0.3729903536977492
ParseFunc = OneHot, K = 2, DisFunc = DisCosine
Classification Accuracy:  0.3954983922829582
ParseFunc = OneHot, K = 3, DisFunc = Dis1
Classification Accuracy:  0.37942122186495175
ParseFunc = OneHot, K = 3, DisFunc = Dis2
Classification Accuracy:  0.37942122186495175
ParseFunc = OneHot, K = 3, DisFunc = DisInf
Classification Accuracy:  0.19614147909967847
Par

Classification Accuracy:  0.45980707395498394
ParseFunc = TI-IDF, K = 5, DisFunc = Dis1
Classification Accuracy:  0.3987138263665595
ParseFunc = TI-IDF, K = 5, DisFunc = Dis2
Classification Accuracy:  0.2508038585209003
ParseFunc = TI-IDF, K = 5, DisFunc = DisInf
Classification Accuracy:  0.34726688102893893
ParseFunc = TI-IDF, K = 5, DisFunc = DisCosine
Classification Accuracy:  0.4437299035369775
ParseFunc = TI-IDF, K = 6, DisFunc = Dis1
Classification Accuracy:  0.3987138263665595
ParseFunc = TI-IDF, K = 6, DisFunc = Dis2
Classification Accuracy:  0.2315112540192926
ParseFunc = TI-IDF, K = 6, DisFunc = DisInf
Classification Accuracy:  0.36977491961414793
ParseFunc = TI-IDF, K = 6, DisFunc = DisCosine
Classification Accuracy:  0.45016077170418006
ParseFunc = TI-IDF, K = 7, DisFunc = Dis1
Classification Accuracy:  0.40514469453376206
ParseFunc = TI-IDF, K = 7, DisFunc = Dis2
Classification Accuracy:  0.21543408360128619
ParseFunc = TI-IDF, K = 7, DisFunc = DisInf
Classification Accura

In [24]:
sorted(regressResults.items(), key=lambda kv: 0 if np.isnan(kv[1]) else kv[1] , reverse=True)


[(('TI-IDF', 9, 'DisCosine'), 0.4070312355638696),
 (('TI-IDF', 8, 'DisCosine'), 0.40536424233469237),
 (('TI-IDF', 10, 'DisCosine'), 0.4052144071304416),
 (('TI-IDF', 7, 'DisCosine'), 0.4021557462636966),
 (('TI-IDF', 5, 'DisCosine'), 0.40148953705180057),
 (('TI-IDF', 11, 'DisCosine'), 0.39826986545002735),
 (('TI-IDF', 6, 'DisCosine'), 0.3980207315613325),
 (('TI-IDF', 12, 'DisCosine'), 0.394279061903327),
 (('TI-IDF', 4, 'DisCosine'), 0.39398895043848653),
 (('TI-IDF', 13, 'DisCosine'), 0.38745096442658084),
 (('TI-IDF', 3, 'DisCosine'), 0.382630435864834),
 (('TI-IDF', 14, 'DisCosine'), 0.3808402107477673),
 (('TI-IDF', 15, 'DisCosine'), 0.37698533588772976),
 (('TI-IDF', 16, 'DisCosine'), 0.3756505766161086),
 (('TI-IDF', 17, 'DisCosine'), 0.369415867316325),
 (('TI-IDF', 2, 'DisCosine'), 0.3682461782438488),
 (('TI-IDF', 19, 'DisCosine'), 0.36596319662753746),
 (('TI-IDF', 18, 'DisCosine'), 0.364184832146661),
 (('OneHot', 9, 'DisCosine'), 0.35723121105176353),
 (('OneHot', 7, '

In [25]:
sorted(classifyResults.items(), key=lambda kv: kv[1], reverse=True)

[(('TI-IDF', 10, 'DisCosine'), 0.4855305466237942),
 (('TI-IDF', 11, 'DisCosine'), 0.4855305466237942),
 (('TI-IDF', 13, 'DisCosine'), 0.4790996784565916),
 (('TI-IDF', 12, 'DisCosine'), 0.4694533762057878),
 (('TI-IDF', 7, 'DisCosine'), 0.4630225080385852),
 (('TI-IDF', 8, 'DisCosine'), 0.4630225080385852),
 (('TI-IDF', 4, 'DisCosine'), 0.45980707395498394),
 (('TI-IDF', 9, 'DisCosine'), 0.4533762057877814),
 (('TI-IDF', 6, 'DisCosine'), 0.45016077170418006),
 (('TI-IDF', 14, 'DisCosine'), 0.45016077170418006),
 (('OneHot', 7, 'DisCosine'), 0.44694533762057875),
 (('OneHot', 8, 'DisCosine'), 0.44694533762057875),
 (('OneHot', 9, 'DisCosine'), 0.4437299035369775),
 (('OneHot', 10, 'DisCosine'), 0.4437299035369775),
 (('TI-IDF', 5, 'DisCosine'), 0.4437299035369775),
 (('OneHot', 11, 'DisCosine'), 0.4405144694533762),
 (('OneHot', 12, 'DisCosine'), 0.4405144694533762),
 (('TI-IDF', 1, 'DisCosine'), 0.4405144694533762),
 (('TI-IDF', 2, 'DisCosine'), 0.4405144694533762),
 (('TI-IDF', 15, '

In [26]:
train_filen = 'lab1_data/regression_dataset/train_set.csv'
vaild_filen = 'lab1_data/regression_dataset/validation_set.csv'
test_filen  = 'lab1_data/regression_dataset/test_set.csv'

ReadFileFunc = regressReadFile
ParseYFunc = regressParseY
TrainFunc = do_regress

ParseFunc = KNN_getTFIDF
DisFunc = DisCosine
K = 9

trainX_data, trainY_data = ReadFileFunc(train_filen)
vaildX_data, vaildY_data = ReadFileFunc(vaild_filen)
word_dict = OrderedDict()
def ParseXFunc(fdata: List[List]): return ParseFunc(fdata, word_dict)
trainX, trainY = vectorizeData(trainX_data, trainY_data, ParseXFunc, ParseYFunc)
vaildX, vaildY = vectorizeData(vaildX_data, vaildY_data, ParseXFunc, ParseYFunc)
def knnFunc(trainX, trainY, vaildX): return KNN((trainX, trainY), vaildX, DisFunc, K, DisInvNormAvg)
ret = TrainFunc(trainX, trainY, vaildX, vaildY, knnFunc, True)

Correlation Coefficient:  0.4070312355638696
